In [1]:
import ktrain

In [3]:
predictor = ktrain.load_predictor("distilbert_pos_neg_3_is_pos")

In [4]:
predictor.explain("this place is seriously the best place i've ever been to!")

In [5]:
predictor = ktrain.load_predictor("distilbert_regression_512/")

In [6]:
predictor.predict("this is a amazing!")

/Users/spenceradams/.local/share/virtualenvs/Yelp-ds-xpajUQwb/lib/python3.7/site-packages/ktrain/text/preprocessor.py:508: UserWarning: Task is being treated as TEXT REGRESSION because class_names argument was not supplied. If this is incorrect, supply class_names argument.
  'class_names argument was not supplied. ' + \


5.0553436

In [7]:
import pandas as pd

In [8]:
test = pd.read_csv("s3://yelp-dataset-pt-9/spencer/data/sentiment/en/test_small.csv")

In [9]:
test.head()

,text,stars,pos_neg_neu,pos_neg_3_is_pos,pos_neg_3_is_neg
0,this club is really pretty and played good mus...,4.0,POSITIVE,POSITIVE,POSITIVE
1,Took my staff out for Lunch. We had a great t...,5.0,POSITIVE,POSITIVE,POSITIVE
2,"I like this place! Tasty food, delicious drink...",5.0,POSITIVE,POSITIVE,POSITIVE
3,Best chain ever for fresh squeezed juices and ...,5.0,POSITIVE,POSITIVE,POSITIVE
4,Amazing place!! Definitely super busy during w...,5.0,POSITIVE,POSITIVE,POSITIVE


In [10]:
from tqdm.notebook import tqdm
tqdm.pandas()

In [11]:
from transformers import AutoTokenizer, AutoConfig, TFAutoModelForSequenceClassification

In [12]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased', use_fast=True)
model = TFAutoModelForSequenceClassification.from_pretrained("distilbert_regression_512/")

In [13]:
import tensorflow as tf

In [14]:
%%timeit
model(tokenizer.encode("this is amazing!", return_tensors='tf'))[0].numpy()[0][0]

112 ms ± 10.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
%%timeit
predictor.predict("this is amazing!")

/Users/spenceradams/.local/share/virtualenvs/Yelp-ds-xpajUQwb/lib/python3.7/site-packages/ktrain/text/preprocessor.py:508: UserWarning: Task is being treated as TEXT REGRESSION because class_names argument was not supplied. If this is incorrect, supply class_names argument.
  'class_names argument was not supplied. ' + \


761 ms ± 73.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
def predict(text):
    return model(tokenizer.encode(text, return_tensors='tf', max_length=512))[0].numpy()[0][0]

In [17]:
preds = test['text'].sample(100, random_state=42).progress_apply(predict)

In [18]:
from sklearn.metrics import mean_absolute_error

In [19]:
mean_absolute_error(test['stars'].sample(100, random_state=42), preds)

0.32072129249572756

In [20]:
predict("this is amazing!")

5.039273

In [21]:
predictor.predict("this is amazing!")

/Users/spenceradams/.local/share/virtualenvs/Yelp-ds-xpajUQwb/lib/python3.7/site-packages/ktrain/text/preprocessor.py:508: UserWarning: Task is being treated as TEXT REGRESSION because class_names argument was not supplied. If this is incorrect, supply class_names argument.
  'class_names argument was not supplied. ' + \


5.039273